# 1) Устанавливаем библиотеки для работы с моделями и атакующими фреймворками

1. **TensorFlow** - Главный фреймворк для машинного обучения от Google.

Создание, обучение и развертывание моденей глубокого обучения (нейронных сетей).


2. **PyTorch + TorchVision** - Основной фреймворк от Facebook.

Создание и обучение нейронных сетей.


3. **torchvision** - Библиотека-компаньон для PyTorch.

Работа с компьютерным зрением.


4. **Foolbox** - Библиотека для проведения adversarial атак (атак на нейронные сети)

Быстрое тестирование и оценка уязвимости моделей к специально созданным "враждебным" примерам, которые обманывают модель.


5. **CleverHans** - Еще одна известная библиотека для исследований в области adversarial машинного обучения

Аналогично Foolbox - реализация атак и методов защиты (обучения устойчивых моделей).





In [2]:
!pip install tensorflow
!pip install torch torchvision
!pip install foolbox
!pip install cleverhans


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.0 MB/s eta 0:00:00


# 2. Проверка установки библиотек

In [3]:
import tensorflow as tf
import torch
import foolbox
import cleverhans

# Проверка версий TensorFlow, PyTorch, Foolbox, CleverHans
print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"Foolbox version: {foolbox.__version__}")
print(f"CleverHans version: {cleverhans.__version__}")

TensorFlow version: 2.19.0
PyTorch version: 2.8.0+cu126
Foolbox version: 3.3.4
CleverHans version: 4.0.0-20324d862500a5ca9ecd00e74b78e1d3


# 3. Загрузка и обучение простой модели

1. Загружаются данные рукописных цифр (60k для обучения, 10k для теста)

2. Данные подготавливаются (нормализуются и преобразуются)

3. Создается простая нейросеть из трех слоев

4. Модель обучается распознавать цифры

5. Оценивается качество работы на новых данных

6. Результат: модель с точностью ~97-98% для распознавания цифр.

In [5]:
# Импорт необходимых компонентов из библиотеки TensorFlow/Keras
from tensorflow.keras.datasets import mnist          # Загрузка датасета MNIST
from tensorflow.keras.models import Sequential       # Линейная модель (последовательные слои)
from tensorflow.keras.layers import Dense, Flatten   # Полносвязный слой и слой выравнивания
from tensorflow.keras.utils import to_categorical    # Преобразование меток в one-hot формат

# Загрузка датасета MNIST и разделение на обучающую и тестовую выборки
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных: преобразование значений пикселей из диапазона [0,255] в [0,1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование числовых меток (0-9) в one-hot encoding (например, 3 -> [0,0,0,1,0,0,0,0,0,0])
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# Создание последовательной модели нейронной сети
model = Sequential([
    Flatten(input_shape=(28, 28)),  # Выравнивание изображения 28x28 в вектор из 784 элементов
    Dense(128, activation='relu'),  # Полносвязный слой с 128 нейронами и функцией активации ReLU
    Dense(10, activation='softmax') # Выходной слой с 10 нейронами (по одному на каждую цифру) и softmax
])

# Компиляция модели: настройка процесса обучения
model.compile(optimizer='adam',                    # Оптимизатор Adam для обновления весов
              loss='categorical_crossentropy',     # Функция потерь для многоклассовой классификации
              metrics=['accuracy'])                # Метрика для оценки - точность

# Обучение модели на тренировочных данных в течение 5 эпох
model.fit(train_images, train_labels, epochs=5)

# Оценка качества модели на тестовых данных, которые она не видела при обучении
test_loss, test_acc = model.evaluate(test_images, test_labels)

print(f"Test accuracy: {test_acc}")

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8795 - loss: 0.4242
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9658 - loss: 0.1162
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9760 - loss: 0.0791
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9842 - loss: 0.0545
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9876 - loss: 0.0425
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9715 - loss: 0.0940
Test accuracy: 0.9760000109672546


In [8]:
#Сохранение модели
model.save('pr1_model.h5')